In [1]:
#General Imports
import os 
import sys
from dotenv import load_dotenv #importing env file
import pandas as pd
import random
import pickle



In [2]:
#ML Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [3]:
#NLP
import gensim
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [4]:
train_path = '/Users/vishesh/projects/Pricer/data/balanced/train.pkl'
test_path = '/Users/vishesh/projects/Pricer/data/balanced/test.pkl'
with open(train_path,'rb') as f:
    train = pickle.load(f)
with open(test_path,'rb') as f:
    test = pickle.load(f)

In [5]:
print(train['title'].iloc[232344])

Starfilm Screen Protector For Thinkpad X1 Yoga Gen6/2021 Touch Screen Laptop 14Inch Draw&Write Like Paper Screen Protector (14inch 16:10 draw like paper)


In [6]:

print(train['prompt'].iloc[3])
print(test['test_prompt'].iloc[3])

How much does this cost to the nearest dollar?

Upgraded Charging Dock with LED Light for Oculus Quest Station with 2 Rechargeable Batteries for Meta Oculus Quest 2 Accessories,Headset Display Stand and Controller Mount,Magnetic Cable
Magetic Charing Station for OCULUS QUEST 2 Charging Dock Built to charge the Oculus Quest 2, the Elite Strap, VR headset and two Touch Controllers simultaneously. Lightning Speed Charge Great for Elite Gamer This charger dock for valve index allows you to fully charge up the both controllers within 4 hours, and fully charge up VR headset with 3.5 hours. There are 3 USB-A ports on the dock meet your needs to charge other devices at the same time. Save Energy and More Environmentally Rechargeable batteries and controller cover help to save you from the endless cycle of replacements. Space Neat and Easy Get for Oculus quest 2 accessories VR stand stores your VR headset and controllers in

Price is $45.00
How much does this cost to the nearest dollar?

Vangoa

In [ ]:
import math
class Tester:

    def __init__(self,i):
        datapoint = self.data.iloc[i]
        self.predictor = predictor 

    def color_for(self,error,truth):
        #defining what I will call as a good guess and color coding for better visibility
        if relative_error < 0.15:
            return "green"
        elif relative_error < 0.3:
            return "yellow"
        else :
            return "red"
        
    def run_datapoint(self ,i):
        #guess and error calculation
        datapoint = self.data[i]
        guess = self.predictor(datapoint)
        truth = datapoint["price"] 
        #error metrics
        relative_error = abs(guess-truth)/truth 
        percentage_error = (relative_error)*100 #percentage error
        #mean absolute percentage error - blows at truth = 0
        mape_component = abs(truth-guess) / truth
        #symmetric mape - solves mape blowing
        smape_component = abs(truth-guess) / ((truth + guess)/2)

        #log(cosh(error))-> how much % the model is off ->best measure ithink
        log_cosh_error = math.log(math.cosh(guess-truth))


34.99